In [1]:
!pip install --upgrade --quiet google.genai


In [2]:
!pip install gTTS --quiet


In [3]:
!pip install playsound


  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=6cb6bd25226a865e78815ac6bf19335881752f880f1bcfe04e3a5f52cc0ae8c3
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [4]:
import google.generativeai as genai


In [5]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get("AGENTIC_AI_KEY")
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")


API Key found


In [6]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model ="gemini-2.0-flash-exp"


In [7]:
video_file_name = "/content/Introduction_video.mp4"


In [9]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)


Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/pk6ris57e5fv


In [12]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video in English, once done then describe the setting of the video and also whatever is said in the video translate it into Spanish.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)
print(response.text)


```json
[
  {
    "timecode": "0:00",
    "caption": "A woman with dark, shoulder-length hair and a pink sweater speaks directly to the camera. She is in a room with a white wall, dark curtains, a small potted plant, a bed with a patterned blanket, and a white chair in the background. She says, \"Hi, my name is Nadia and I'm a PIIC student. It's my first time learning uh technology, but I'm really loving it. I am passionate about um tech field. Though I thought it something that that might be difficult, but I'm enjoying it so far. So thank you very much PIIC team for your help and support, and I think this is going to be a turning point in my life learning and uh...\""
  }
]
```

**Setting Description:**

The video appears to be filmed in a casual home setting. The background includes a bedroom with a bed, a chair and dark curtains. There is also a small houseplant visible. The walls are light-colored.

**Spanish Translation:**

"Hola, mi nombre es Nadia y soy estudiante de PIIC. Es la